In [1]:
import pandas as pd

import numpy as np

import re

In [2]:
config_prefix = ['TI', 'TR', 'BR', 'BI']

config_algo_names = ['BinarySearch', 'BubbleSort', 'Factorial', 'Fibonacci', 'IntegerBinary', 'MultiplyMatrix', 'PrimeFactors', 'ReverseString']

config_id_variable = "CASE"

config_answer_variables = [
    ['TI21_01', 'TR21_01', 'BR21_01', 'BI21_01'],
    ['TI22_01', 'TR22_01', 'BR22_01', 'BI22_01'],
    ['TI23_01', 'TR23_01', 'BR23_01', 'BI23_01'],
    ['TI24_01', 'TR24_01', 'BR24_01', 'BI24_01'],
    ['TI25_01', 'TR25_01', 'BR25_01', 'BI25_01'],
    ['TI26_01', 'TR26_01', 'BR26_01', 'BI26_01'],
    ['TI27_01', 'TR27_01', 'BR27_01', 'BI27_01'],
    ['TI28_01', 'TR28_01', 'BR28_01', 'BI28_01'],
]

config_time_variables = [
    ['TIME011','TIME083','TIME035','TIME059'],
    ['TIME038','TIME014','TIME062','TIME086'],
    ['TIME065','TIME041','TIME086','TIME017'],
    ['TIME092','TIME068','TIME020','TIME044'],
    ['TIME023','TIME095','TIME047','TIME071'],
    ['TIME050','TIME026','TIME074','TIME098'],
    ['TIME077','TIME053','TIME0101','TIME029'],
    ['TIME0104','TIME080','TIME032','TIME056'],
]



config_answer_patterns = [
    ['.*3.*', '.*3.*', '.*3.*', '.*3.*'],
    ['.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*'],
    ['.*6.*','.*6.*','.*120.*','.*120.*'],
    ['.*2.*', '.*2.*','.*2.*','.*2.*'],
    ['.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*'],
    ['.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*'],
    ['.*3.*5.*','.*3.*5.*','.*3.*5.*','.*3.*5.*'],
    ['.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*']
]


config_datasheet = r'./data_DG.xlsx'

In [3]:
raw = pd.read_excel(config_datasheet)


from scipy.stats import ttest_ind



In [4]:
df_matrix = []
raw = pd.read_excel(config_datasheet)

# load data
for row, _stuff in enumerate(config_algo_names):
    df_array = []
    for col, _stuff in enumerate(config_prefix):
        df = pd.DataFrame(raw, columns = [config_id_variable, 
                                          config_answer_variables[row][col], 
                                          config_time_variables[row][col],])
        df = df.iloc[1:]
        df = df.dropna()
        df_array.append(df)
        
        
    df_matrix.append(df_array)

In [5]:
cols = ["id", "response_time", "correctness", "flag", "algo_name"]
df_BU_R = pd.DataFrame(columns=cols)
df_BU_I = pd.DataFrame(columns=cols)
df_TD_R = pd.DataFrame(columns=cols)
df_TD_I = pd.DataFrame(columns=cols)

for row_idx, df_row in enumerate(df_matrix):
    algo_name = config_algo_names[row_idx]
    for col, df in enumerate(df_row): 
        pattern = config_answer_patterns[row_idx][col]
        regex = re.compile(pattern)
        
        for index, row in df.iterrows():

            id_value =  row["CASE"]
            response_time = int(row[config_time_variables[row_idx][col]])
            answer = row[config_answer_variables[row_idx][col]]
            result = regex.match(str(answer))
            correctness = False
            
            if result is not None:
                correctness = True
                
            flag = ""
            data = pd.DataFrame([[id_value, response_time, correctness, flag, algo_name]], columns=cols)
                  
            if col == 0:
                df_BU_R = df_BU_R.append(data)
            elif col == 1:
                df_BU_I = df_BU_I.append(data)
            elif col == 2:
                df_TD_R = df_TD_R.append(data)
            else:
                df_TD_I = df_TD_I.append(data)

In [6]:
df_array_independent = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]

In [7]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
cols = ["Subject","ProgrammingStyle", "Comprehension", "ResponseTime", "Algorithm", "Flag", "Correctness" ]

def fill(list_data, dataframe, coding_style, comprehension_style):
    for idx, row in dataframe.iterrows():
        response_time = row["response_time"]
        algorithm = row["algo_name"]
        flag = row["flag"]
        correctness = row["correctness"]
        id = row["id"]
        list_data.append([id,coding_style, comprehension_style, response_time, algorithm, flag, correctness])

data = []
fill(data, df_BU_R, "R", "BU")
fill(data, df_BU_I, "I", "BU")
fill(data, df_TD_R, "R", "TD")
fill(data, df_TD_I, "I", "TD")
df = pd.DataFrame(data, columns=cols)
print (df)
df.to_excel("./results/preprocessed.xlsx", index=False)

     Subject ProgrammingStyle Comprehension  ResponseTime      Algorithm Flag  \
0      198.0                R            BU             8   BinarySearch        
1      211.0                R            BU            58   BinarySearch        
2      224.0                R            BU            30   BinarySearch        
3      231.0                R            BU            85   BinarySearch        
4      232.0                R            BU             5   BinarySearch        
..       ...              ...           ...           ...            ...  ...   
128    182.0                I            TD             4  ReverseString        
129    210.0                I            TD            72  ReverseString        
130    212.0                I            TD            60  ReverseString        
131    226.0                I            TD            29  ReverseString        
132    234.0                I            TD           113  ReverseString        

     Correctness  
0       